In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = r'T:\toandangquoc\iuh_project_finals\data\data_train_model.csv'

In [10]:
df = pd.read_csv(path)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.head()

,high,low,open,unix,cum_change,sma_7_up,sma_7,sma_7_low,ema_14,b_ma,bu,bl,cma_7,month,atr,year,ppo,close
date,,,,,,,,,,,,,,,,,,
2020-01-02,209.789993,206.270004,206.750000,1577923200,0.045198,210.794776,209.779999,207.655221,209.779999,217.042583,227.120132,206.965034,211.514287,1,3.519989,2020,0.000000,209.779999
2020-01-03,210.399994,206.949997,207.210007,1578009600,0.090156,210.794776,209.224998,207.655221,209.631999,217.042583,227.120132,206.965034,211.514287,1,3.514990,2020,-0.042226,208.669998
2020-01-04,211.193329,206.806666,207.040003,1578096000,0.135397,210.888098,209.476666,208.065234,209.678399,217.042583,227.120132,206.965034,211.514287,1,3.577252,2020,-0.024991,209.980001
2020-01-05,211.986664,206.663335,206.870000,1578182400,0.180920,212.078555,209.930000,207.781446,209.893280,217.042583,227.120132,206.965034,211.514287,1,3.701972,2020,0.038602,211.290003
2020-01-06,212.779999,206.520004,206.699997,1578268800,0.226725,213.491436,210.464001,207.436567,210.254176,217.042583,227.120132,206.965034,211.514287,1,3.884688,2020,0.137642,212.600006


### ***Sarimax***

In [11]:
split_ratio = 0.8
cut_off = int(len(df) * split_ratio)
train = df[:cut_off]
test = df[cut_off:]
X_train = train.drop(['close'], axis=1)
X_test = test.drop(['close'], axis=1)
y_train = train['close']
y_test = test['close']
print("Kích thước của X_train: ", X_train.shape)
print("Kích thước của y_train: ", y_train.shape)
print("Kích thước của X_test: ", X_test.shape)
print("Kích thước của y_test: ", y_test.shape)

Kích thước của X_train:  (1338, 17)
Kích thước của y_train:  (1338,)
Kích thước của X_test:  (335, 17)
Kích thước của y_test:  (335,)


### ***LSTM***

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
split_ratio=0.8
cut_off = int(len(df) * split_ratio)
train = df[:cut_off]
test = df[cut_off:]
scaler = MinMaxScaler(feature_range=(0, 1))

In [14]:
scaled_train = scaler.fit_transform(train)
print('Train shape:', scaled_train.shape)
scaled_test = scaler.fit_transform(test)
print('Test shape:', scaled_test.shape)
X_train = []
y_train = []
X_test = []
y_test = []
test_length = len(df) - cut_off
prediction_step = 50
for i in range(prediction_step, cut_off):
    X_train.append(scaled_train[i - prediction_step:i, 0:scaled_train.shape[1]])
    y_train.append(scaled_train[i, -1])
X_train, y_train = np.array(X_train), np.array(y_train)
y_train = np.reshape(y_train, (y_train.shape[0], 1))
print('X_train shape :', X_train.shape)
print('y_train shape :', y_train.shape)

for i in range(prediction_step, test_length):
    X_test.append(scaled_test[i - prediction_step:i, 0:scaled_test.shape[1]])
    y_test.append(scaled_test[i, -1])
X_test, y_test = np.array(X_test), np.array(y_test)
y_test = np.reshape(y_test, (-1, 1))
        
print('X_test shape :', X_test.shape)
print('y_test shape :', y_test.shape)

Train shape: (1338, 18)
Test shape: (335, 18)
X_train shape : (1288, 50, 18)
y_train shape : (1288, 1)
X_test shape : (285, 50, 18)
y_test shape : (285, 1)


### ***Xgboots***

In [15]:
cut_off = int(len(df) * split_ratio)
train = df[:cut_off]
test = df[cut_off:]
train_scale = scaler.fit_transform(train)
test_scale = scaler.fit_transform(test)
X = df.iloc[:, :-1]
y = df['close']
X_train, y_train = train_scale[:, :-1], train_scale[:, -1].reshape(-1, 1)
X_test, y_test = test_scale[:, :-1], test_scale[:, -1].reshape(-1, 1)
print('X_train shape :', X_train.shape)
print('y_train shape :', y_train.shape)      
print('X_test shape :', X_test.shape)
print('y_test shape :', y_test.shape)

X_train shape : (1338, 17)
y_train shape : (1338, 1)
X_test shape : (335, 17)
y_test shape : (335, 1)


### ***RandomForest***

In [16]:
def prepare_sequences( data, prediction_step=50):
        """Prepare sequential input for time series prediction."""
        X, y = [], []
        for i in range(prediction_step, len(data)):
            X.append(data.iloc[i-prediction_step:i].values.flatten())
            y.append(data.iloc[i]['close'])
        return np.array(X), np.array(y)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X, y = prepare_sequences(df)
        
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
            X, y, train_size=split_ratio, shuffle=False
)
        
        # Scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
        
print('X_train shape:', X_train_scaled.shape)
print('X_test shape:', X_test_scaled.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (1298, 900)
X_test shape: (325, 900)
y_train shape: (1298,)
y_test shape: (325,)
